In [24]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# 파일 불러오기

In [64]:
# 파일 경로 설정
score_file_path = '/home/woonj/grad-cam/experiment_results/grad_cam_bonafide_score.txt'
vad_directory_path = '/home/woonj/grad-cam/PartialSpoof/database/vad_20sil/eval'

In [65]:
# grad_cam_spoof_score 파일 읽기
grad_cam_data = {}
with open(score_file_path, 'r') as f:
    for line in f.readlines()[1:]:  # 첫 번째 라인은 헤더이므로 건너뜀
        file_id, _, cam_values_str = line.strip().split(' ')
        cam_values = np.array([float(x) for x in cam_values_str.split(',')])
        grad_cam_data[file_id] = cam_values

# RCQ 구하기

### 전처리

In [66]:
# 각 구간을 위한 점수 집계 변수 초기화
total_scores = {'TR': 0, 'BS': 0, 'BN': 0, 'SS': 0, 'SN': 0}
frame_counts = {'TR': 0, 'BS': 0, 'BN': 0, 'SS': 0, 'SN': 0}

In [67]:
# 라벨 ID와 구간 이름 매핑
label_mapping = {
    0: 'BN',        # Bona fide Non-speech
    1: 'BS',        # Bona fide Speech
    2: 'SS', 3: 'SS', 4: 'SS', 5: 'SS', 6: 'SS', 7: 'SS', 8: 'SS', 9: 'SS', 10: 'SS',
    11: 'SS', 12: 'SS', 13: 'SS', 14: 'SS', 15: 'SS', 16: 'SS', 17: 'SS', 18: 'SN', 19: 'SS', 20: 'SS', # Spoofed Speech 구간들 (A01 ~ A19)
    100: 'TR',      # Transition Region
    101: 'BN',      # Bona fide Non-speech (특정 타입)
    102: 'SN', 103: 'SN', 104: 'SN', 105: 'SN', 106: 'SN', 107: 'SN', 108: 'SN', 109: 'SN', 110: 'SN',
    111: 'SN', 112: 'SN', 113: 'SN', 114: 'SN', 115: 'SN', 116: 'SN', 117: 'SN', 118: 'SN', 119: 'SN', 120: 'SN'  # Spoofed Non-speech (nonA01 ~ nonA19)
}

### RCQ 값 구하기

In [68]:
# vad 파일 목록에서 확장자 제거
vad_files = [os.path.splitext(file)[0] for file in os.listdir(vad_directory_path) if file.endswith('.vad')]

# 각 파일에 대해 VAD 정보를 읽어 Grad-CAM 점수와 매칭
for vad_file in tqdm(vad_files, desc="Processing VAD files"):
    # grad-cam 점수가 없으면 건너뛰기
    if vad_file not in grad_cam_data:
        print("vad_file not in grad_cam_data")
        continue

    # Grad-CAM 점수 로드
    grad_cam_scores = grad_cam_data[vad_file] # 현재 vad_file에 대응하는 점수 저장
    num_frames = len(grad_cam_scores) # 총 프레임 수 계산
    frame_length = 0.02  # 각 프레임의 길이 (단위: 초) 20ms

    # VAD 파일 경로 설정
    vad_file_path = os.path.join(vad_directory_path, f"{vad_file}.vad")

    # VAD 파일 읽기
    vad_labels = [] # 구간 정보를 저장할 배열
    with open(vad_file_path, 'r') as file:
        for line in file:
            start_time, end_time, label_id = line.strip().split()
            vad_labels.append((float(start_time), float(end_time), int(label_id)))

    # 각 프레임에 대한 라벨 할당
    for frame_idx in tqdm(range(num_frames), desc=f"Processing frames for {vad_file}", leave=False):
        frame_start_time = frame_idx * frame_length # 현재 프레임의 시작 시간
        frame_end_time = (frame_idx + 1) * frame_length # 현재 프레임의 끝 시간
        label_durations = {} # 각 라벨이 겹치는 구간을 저장

        # 해당 프레임이 속하는 모든 VAD 구간 확인
        for (start_time, end_time, label_id) in vad_labels:
            overlap_start = max(frame_start_time, start_time) # 겹치는 시간 계산 -> 프레임 시작 시간과 VAD 구간 시간 중 더 큰 값
            overlap_end = min(frame_end_time, end_time) # 프레임 끝 시간과 VAD 구간 끝 시간중 더 작은 값
            if overlap_start < overlap_end:  # 프레임과 VAD 구간이 겹치는 경우
                overlap_duration = overlap_end - overlap_start # 겹치는 구간의 길이를 계산
                label_name = label_mapping.get(label_id, None) # 라벨 id를 매핑하여 구간 이름 가져오기 -> 매핑 되지 않으면 None

                if label_name:
                    if label_name not in label_durations: # 겹치는 구간 딕셔너리에 해당 라벨이 없으면 초기화한다.
                        label_durations[label_name] = 0
                    label_durations[label_name] += overlap_duration

        # 가장 긴 시간을 차지하는 라벨을 선택
        if label_durations:
            assigned_label = max(label_durations, key=label_durations.get) # 겹치는 시간이 가장 긴 라벨을 선택

            # 구간에 맞는 라벨이 있는 경우 점수 집계
            if assigned_label in total_scores:
                total_scores[assigned_label] += grad_cam_scores[frame_idx]
                frame_counts[assigned_label] += 1

Processing VAD files: 100%|██████████| 71237/71237 [29:26<00:00, 40.32it/s]


In [69]:
# 각 구간의 평균 Grad-CAM 점수 계산
# target_class = bonafide
average_scores = {label: total_scores[label] / frame_counts[label] if frame_counts[label] > 0 else 0 for label in total_scores}
print(average_scores)

{'TR': 5.406169090358454e-05, 'BS': 0.00016001258435394624, 'BN': 0.00011696894864430369, 'SS': -9.475556798409995e-05, 'SN': -9.039189209921813e-06}


In [72]:
# 전체 평균 Grad-CAM 점수 계산
# target_class = bonafide
overall_average_score = sum(total_scores.values()) / sum(frame_counts.values())

# 각 구간에 대한 RCQ 계산
rcq_values = {}
for label, avg_score in average_scores.items():
    rcq_values[label] = ((avg_score - overall_average_score) / abs(overall_average_score)) * 100

# RCQ 값 출력
print("RCQ 값:")
for label, rcq in rcq_values.items():
    print(f"구간 {label}: {rcq:.2f}%")

RCQ 값:
구간 TR: -20.71%
구간 BS: 134.68%
구간 BN: 71.55%
구간 SS: -238.97%
구간 SN: -113.26%


In [63]:
# 각 구간의 평균 Grad-CAM 점수 계산
# target_class = spoof
average_scores = {label: total_scores[label] / frame_counts[label] if frame_counts[label] > 0 else 0 for label in total_scores}
print(average_scores)

{'TR': -5.445364061363494e-05, 'BS': -0.00016066983568008804, 'BN': -0.00011731160405126448, 'SS': 9.406910362516034e-05, 'SN': 8.753507511852358e-06}


In [62]:
# 전체 평균 Grad-CAM 점수 계산
# target_class = spoof
overall_average_score = sum(total_scores.values()) / sum(frame_counts.values())

# 각 구간에 대한 RCQ 계산
rcq_values = {}
for label, avg_score in average_scores.items():
    rcq_values[label] = ((avg_score - overall_average_score) / overall_average_score) * 100

# RCQ 값 출력
print("RCQ 값:")
for label, rcq in rcq_values.items():
    print(f"구간 {label}: {rcq:.2f}%")

RCQ 값:
구간 TR: -20.77%
구간 BS: 133.77%
구간 BN: 70.68%
구간 SS: -236.87%
구간 SN: -112.74%


In [48]:
# 파일 경로 설정
score_file_path = '/home/woonj/grad-cam/experiment_results/grad_cam_spoof_score.txt'

# 파일 ID 설정 (VAD 파일과 동일한 파일 ID 사용)
file_id = 'CON_E_0009987'

# Grad-CAM 점수 파일 읽기 및 해당 파일의 점수 추출
grad_cam_scores = None
print("Grad-CAM 점수 파일을 읽는 중...")
with open(score_file_path, 'r') as f:
    for line in f.readlines()[1:]:  # 첫 번째 라인은 헤더이므로 건너뜀
        current_file_id, _, cam_values_str = line.strip().split(' ')
        if current_file_id == file_id:
            cam_values_list = cam_values_str.split(',')
            grad_cam_scores = np.array([float(x) for x in cam_values_list])
            break

# 점수를 찾지 못한 경우 오류 출력
if grad_cam_scores is None:
    raise ValueError(f"{file_id}에 대한 Grad-CAM 점수를 찾을 수 없습니다.")
print(f"{file_id}에 대한 Grad-CAM 점수 로드 완료 - 총 {len(grad_cam_scores)}개의 프레임 점수")

# VAD 파일 내용 (예시)
vad_content = """
0.0 0.0323 117
0.0323 0.04 100
0.04 0.062200000000000005 101
0.0622 0.2422 1
0.2422 0.5159 101
0.5159 0.5217 100
0.5218 0.5285000000000001 117
0.5284 0.5285 101
0.5285 0.9285 1
0.9285 0.9311 101
0.9311 0.9335 100
0.9335 0.9411 117
0.9411 1.4311 17
1.4311 1.8591 117
"""
print("VAD 파일 로드 완료")

# 라벨 ID와 구간 이름 매핑
label_mapping = {
    0: 'BN', 1: 'BS',
    17: 'SS', 100: 'TR', 101: 'BN',
    117: 'SN'
}

# VAD 데이터를 파싱하여 리스트로 저장
vad_labels = []
for line in vad_content.strip().split('\n'):
    start_time, end_time, label_id = line.strip().split()
    vad_labels.append((float(start_time), float(end_time), int(label_id)))

print(f"총 {len(vad_labels)}개의 VAD 라벨 구간 로드 완료")

# 프레임 길이 설정
frame_length = 0.02  # 각 프레임의 길이 (단위: 초)

# 오디오 전체 길이 계산
total_audio_length = max(end_time for _, end_time, _ in vad_labels)
print(f"오디오 전체 길이: {total_audio_length:.4f}초")

# 총 프레임 수 계산
num_frames = int(np.ceil(total_audio_length / frame_length))
num_frames = min(num_frames, len(grad_cam_scores))  # grad_cam_scores 길이와 일치하도록 조정
print(f"총 프레임 수: {num_frames} 프레임")

# 각 구간별 점수 집계 및 프레임 개수를 세기 위한 초기화
total_scores = {'TR': 0, 'BS': 0, 'BN': 0, 'SS': 0, 'SN': 0}
frame_counts = {'TR': 0, 'BS': 0, 'BN': 0, 'SS': 0, 'SN': 0}

# 각 프레임에 대한 라벨 할당 및 점수 집계
print("각 프레임에 대해 라벨 할당 및 점수 집계 시작...")
for frame_idx in range(num_frames):
    frame_start_time = frame_idx * frame_length
    frame_end_time = (frame_idx + 1) * frame_length
    assigned_label = None
    label_durations = {}

    # 해당 프레임이 속하는 모든 VAD 구간 확인
    for (start_time, end_time, label_id) in vad_labels:
        overlap_start = max(frame_start_time, start_time)
        overlap_end = min(frame_end_time, end_time)
        if overlap_start < overlap_end:  # 프레임과 VAD 구간이 겹치는 경우
            overlap_duration = overlap_end - overlap_start
            label_name = label_mapping.get(label_id, None)

            if label_name:
                if label_name not in label_durations:
                    label_durations[label_name] = 0
                label_durations[label_name] += overlap_duration

    # 가장 긴 시간을 차지하는 라벨을 선택
    if label_durations:
        assigned_label = max(label_durations, key=label_durations.get)

    # 라벨이 할당된 경우, 해당 구간의 점수 집계 및 프레임 개수 증가
    if assigned_label is not None:
        total_scores[assigned_label] += grad_cam_scores[frame_idx]
        frame_counts[assigned_label] += 1

        # 각 프레임 처리 상태 출력
        print(f"프레임 {frame_idx + 1}/{num_frames}: 라벨 {assigned_label}, 점수 {grad_cam_scores[frame_idx]}")

print("프레임 점수 집계 완료")

# 각 구간의 평균 Grad-CAM 점수 계산
average_scores = {label: (total_scores[label] / frame_counts[label]) if frame_counts[label] > 0 else 0 for label in total_scores}
print("각 구간의 평균 Grad-CAM 점수 계산 완료")
for label, avg_score in average_scores.items():
    print(f"구간 {label}: 평균 점수 {avg_score}")

# 전체 평균 Grad-CAM 점수 계산
overall_average_score = sum(total_scores.values()) / sum(frame_counts.values())
print(f"전체 평균 Grad-CAM 점수: {overall_average_score}")

# 각 구간에 대한 RCQ 계산
rcq_values = {}
for label, avg_score in average_scores.items():
    rcq_values[label] = ((avg_score - overall_average_score) / overall_average_score) * 100

# RCQ 값 출력
print("RCQ 값:")
for label, rcq in rcq_values.items():
    print(f"구간 {label}: {rcq:.2f}%")


Grad-CAM 점수 파일을 읽는 중...
CON_E_0009987에 대한 Grad-CAM 점수 로드 완료 - 총 92개의 프레임 점수
VAD 파일 로드 완료
총 14개의 VAD 라벨 구간 로드 완료
오디오 전체 길이: 1.8591초
총 프레임 수: 92 프레임
각 프레임에 대해 라벨 할당 및 점수 집계 시작...
프레임 1/92: 라벨 SN, 점수 3.9745457e-05
프레임 2/92: 라벨 SN, 점수 3.9645915e-05
프레임 3/92: 라벨 BN, 점수 3.8910184e-05
프레임 4/92: 라벨 BS, 점수 3.921593e-05
프레임 5/92: 라벨 BS, 점수 3.942642e-05
프레임 6/92: 라벨 BS, 점수 4.004636e-05
프레임 7/92: 라벨 BS, 점수 3.9369916e-05
프레임 8/92: 라벨 BS, 점수 3.913652e-05
프레임 9/92: 라벨 BS, 점수 3.903878e-05
프레임 10/92: 라벨 BS, 점수 3.930866e-05
프레임 11/92: 라벨 BS, 점수 3.9532206e-05
프레임 12/92: 라벨 BS, 점수 3.9170947e-05
프레임 13/92: 라벨 BN, 점수 3.9130668e-05
프레임 14/92: 라벨 BN, 점수 3.899973e-05
프레임 15/92: 라벨 BN, 점수 3.855354e-05
프레임 16/92: 라벨 BN, 점수 3.894885e-05
프레임 17/92: 라벨 BN, 점수 3.8178798e-05
프레임 18/92: 라벨 BN, 점수 3.887451e-05
프레임 19/92: 라벨 BN, 점수 3.9020517e-05
프레임 20/92: 라벨 BN, 점수 3.9415656e-05
프레임 21/92: 라벨 BN, 점수 3.9146355e-05
프레임 22/92: 라벨 BN, 점수 3.8031416e-05
프레임 23/92: 라벨 BN, 점수 3.9548915e-05
프레임 24/92: 라벨 BN, 점수 3.872481e-05
프레임